In [1]:
import sqlite3
import yaml
import pandas as pd
from sklearn.ensemble import IsolationForest

In [2]:
with open('.\configurations.yml') as file:
    configurations = yaml.load(file, Loader=yaml.FullLoader)

db = sqlite3.connect(configurations["Database"]["Path"]) # connecting to sqlite database

df = pd.read_sql_query('''SELECT 
  id,
  valorTransacao,
  dataTransacao,
  codigoSubclasse,
  tipoCodigo,
  ROW_NUMBER() OVER(PARTITION BY valorTransacao, nome_portador ORDER BY dataTransacao ASC) 								AS valorTransacao_count_byPerson,
  ROW_NUMBER() OVER(PARTITION BY codigoFormatado_estabelecimento, nome_portador ORDER BY dataTransacao ASC) 			AS codigoFormatado_estabelecimento_count_byPerson,
  ROW_NUMBER() OVER(PARTITION BY valorTransacao, codigo_unidadeGestora ORDER BY dataTransacao ASC) 						AS valorTransacao_count_byAgency,
  ROW_NUMBER() OVER(PARTITION BY codigoFormatado_estabelecimento, codigo_unidadeGestora ORDER BY dataTransacao ASC) 	AS codigoFormatado_estabelecimento_count_byAgency,
  ROW_NUMBER() OVER(PARTITION BY valorTransacao, nome_orgaoMaximo ORDER BY dataTransacao ASC) 							AS valorTransacao_count_byDepartment,
  ROW_NUMBER() OVER(PARTITION BY codigoFormatado_estabelecimento, nome_orgaoMaximo ORDER BY dataTransacao ASC) 			AS codigoFormatado_estabelecimento_count_bydepartment  
FROM cartoes 
WHERE valorTransacao||nome_estabelecimento||nome_portador not in (SELECT -valorTransacao||nome_estabelecimento||nome_portador from cartoes where valorTransacao <0)
and valorTransacao > 0''', db)

In [3]:
df['dataTransacao'] = pd.to_datetime(df['dataTransacao'])
df['diaSemana'] = df['dataTransacao'].dt.dayofweek
df['dia'] = df['dataTransacao'].dt.day
df['mes'] = df['dataTransacao'].dt.month
df['ano'] = df['dataTransacao'].dt.year

In [4]:
df.codigoSubclasse = df.codigoSubclasse.map(int)

In [5]:
df.tipoCodigo = df.tipoCodigo.apply(lambda x: 1 if x=='CNPJ' else 0)

In [6]:
#df.drop(labels=['id','dataTransacao'],axis=1).head()

In [7]:
model = IsolationForest(random_state=42)
df['result'] = model.fit_predict(df.drop(labels=['id','dataTransacao'],axis=1))
df['score'] = -model.score_samples(df.drop(labels=['id','dataTransacao','result'],axis=1))

In [8]:
df.result.value_counts(normalize=True)

 1    0.873001
-1    0.126999
Name: result, dtype: float64

In [9]:
df[['id','score','result']].to_sql(name='results', con=db, if_exists='replace', index=False)

In [10]:
#df.hist('score')

In [11]:
#df.drop(labels='result',axis=1,inplace=True)